In [30]:
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt 

In [31]:
def charac_poly(a,b):
    n = len(a) + 1
    f, q = np.zeros((2,n))
    f[0] = 1
    for i in range(0, n -1):
        f[i + 1] = a[i] * f[i] 
        if i > 0:
            f[i+1] -= b[i-1]**2 * f[i - 1] 
        if abs(f[i]) < 10**(-15):
            q[i + 1] = a[i] - abs(b[i-1])*10**(-15)
        else:
            q[i +1] = f[i + 1]/f[i]
    return f[-1]

In [32]:
def stable_ver(a,b):
    n = len(a) 
    q = np.zeros(n + 1)
    q[0] = 1
    q[1] = a[0]
    for i in range(1, n):
        q[i+1] = a[i] - b[i-1]**2 / q[i - 1]
    return q[-1]

In [33]:
def bisect(a, b, low, up, max_count, tolerance):
    
    count = 0    
    while np.abs(up - low) > tolerance and count < max_count:
        c = 0.5*(low + up)
        fa = charac_poly(a - low, b)
        fb = charac_poly(a - up, b)
        fc = charac_poly(a - c, b)   
        if fa*fb > 0:
            raise ArithmeticError("Fail at ", count)   
            
        if fa*fc < 0:
            up = c 
        if fc*fb < 0:
            low = c
        count += 1 
    return up
      

In [49]:
def find_eigen(a,b, max_iter = 10**5, max_eigen = 3, tol = 10**-5):
    """
    UNSTABLE VERSION
    calculates the first max_eigen values of a tridiagonal, symetric matrix
    with a the diagonal and b the non diagonal entries
    a is expected to be sorted in ascending order
    max_iter is used for the maximum number of iterations
    returns the first max_eigen eigenvalues
    """
    n = len(a)
    lam = np.zeros(max_eigen + 1)
    print(lam)
    #setup bounderis with Gash Goren
    bounds_low = np.array([a[i] - np.abs(b[i]) - np.abs(b[i+1]) for i in range(max_eigen + 1)])
    bounds_up = np.array([a[i] + np.abs(b[i]) + np.abs(b[i+1]) for i in range(max_eigen + 1)])
    #first iteration lam = a[0]
    bounds_up[0] = a[0]
    bounds_low[1] = a[0]
    for i in range(2,n):
        print(i)
        blt = np.copy(bounds_low)
        but = np.copy(bounds_up)
        lt = np.copy(lam)
        for j in range(max_eigen + 1):
            if j < i:
                lam[j] = bisect(a[:i], b[:i - 1 ], blt[j], but[j], max_count=max_iter, tolerance=tol)
                #update boundaries with interlacing theorem
                bounds_up[j] = lam[j]
                if j + 1 < max_eigen + 1:
                    bounds_low[j + 1] = lam[j]

        if np.max(np.abs(lam-lt)) < tol:
            print('Converged')
            return lam[:-1]
    print(lam[:-1])
    return lam[:-1]

In [50]:
a = np.full(4,2)
b = np.full(4,-1)

A = 2 * np.eye(4,4) - 1 * np.eye(4,4, k = -1) - 1 * np.eye(4,4, k = 1)
print(A)
#print(lam)



eigval, _ = LA.eig(A)
print(eigval)

find_eigen(a,b)

[[ 2. -1.  0.  0.]
 [-1.  2. -1.  0.]
 [ 0. -1.  2. -1.]
 [ 0.  0. -1.  2.]]
[3.61803399 2.61803399 0.38196601 1.38196601]
[0. 0. 0. 0.]


IndexError: index 4 is out of bounds for axis 0 with size 4